In [1]:
%matplotlib widget
from IPython.display import display, Markdown
import os
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model._base import LinearModel
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from rega3 import export_estimator

In [13]:
display(Markdown('### Load Data\n---'))
df_raw = pd.read_csv('ChosunOne_ArtilleryData.csv')
df_raw['distance'] = np.sqrt((df_raw['Target Position X']-df_raw['Firing Position X'])**2 + (df_raw['Target Position Y']-df_raw['Firing Position Y'])**2)
df_raw['height'] = df_raw['Target Elevation'] - df_raw['Firing Elevation']
df = df_raw[['distance', 'height', 'Firing Angle']]
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print('X.shape:', X.shape, sep='\t')
print('X_train.shape:', X_train.shape, sep='\t')
print('X_test.shape:', X_test.shape, sep='\t')
#df

### Load Data
---

X.shape:	(29, 2)
X_train.shape:	(20, 2)
X_test.shape:	(9, 2)


In [14]:
display(Markdown('### Model Selection\n---'))
models = []
polyfeat = PolynomialFeatures()
for linreg in [LinearRegression(), Ridge(normalize=True)]:
    pipe = Pipeline([('polyfeat', polyfeat), ('linreg', linreg)])
    param_grid = {'polyfeat__degree': range(1, 5)}
    if not isinstance(linreg, LinearRegression):
        param_grid['linreg__alpha'] = np.logspace(-10, 0, 50)
    search = GridSearchCV(pipe, param_grid, cv=5, n_jobs=12)
    search.fit(X_train, y_train)
    models.append((search.best_score_, search))
best_model = max(models, key=lambda p: p[0])[1]
print('best_model:', best_model.best_estimator_)

### Model Selection
---

best_model: Pipeline(steps=[('polyfeat', PolynomialFeatures(degree=3)),
                ('linreg',
                 Ridge(alpha=1.2067926406393288e-06, normalize=True))])


In [15]:
display(Markdown('### Test Model\n---'))
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)
print('RMSE train: %.3f, test: %.3f' % (np.sqrt(mean_squared_error(y_train, y_train_pred)), np.sqrt(mean_squared_error(y_test, y_test_pred))))
print('R^2 train: %.3f, test: %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))

### Test Model
---

RMSE train: 0.367, test: 0.911
R^2 train: 0.998, test: 0.992


In [16]:
display(Markdown('### Predict Unknown Point\n---'))
point = [100, 0]
print('Prediction for', point, ':', best_model.predict([point])[0])

### Predict Unknown Point
---

Prediction for [100, 0] : 77.2813525918423


In [17]:
display(Markdown('### Export Estimator\n---'))
function_name = '_test_estimator'
sqf_code = export_estimator(best_model, function_name)
Markdown(f'```c++\n{sqf_code}\nsystemChat format ["Prediction for {point}: %1", {point} call {function_name}];\n```')

### Export Estimator
---

```c++
_test_estimator_polyfeat = {[
    1, (_this#0), (_this#1), (_this#0)^2, (_this#0)*(_this#1), (_this#1)^2,
    (_this#0)^3, (_this#0)^2*(_this#1), (_this#0)*(_this#1)^2, (_this#1)^3]
};

_test_estimator_linreg = {
    -7.594683e-01*(_this#1) - 3.737601e-01*(_this#2) + 4.451657e-03*(_this#3) +
    3.988812e-03*(_this#4) + 4.182825e-05*(_this#5) - 9.980068e-06*(_this#6) -
    1.338557e-05*(_this#7) - 1.669852e-06*(_this#8) + 1.037233e-05*(_this#9) +
    1.186917e+02
};

_test_estimator = {
    _this = _this call _test_estimator_polyfeat;
    _this = _this call _test_estimator_linreg;
    _this
};

systemChat format ["Prediction for [100, 0]: %1", [100, 0] call _test_estimator];
```